<div>
<img src="https://www.nebrija.com/images/logos/logotipo-universidad-nebrija.jpg" width="200">
</div>

# **Prueba Shor Adiabatico**

In [1]:
import dimod
import numpy as np
from dwave.system import LeapHybridCQMSampler
import numpy as np
import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt

In [2]:
N = 13*17
nx = np.ceil(np.log2(np.floor(np.sqrt(N)))) - 1
ny = np.ceil(np.log2(np.floor(N/3))) - 1

total_qubits = nx+ny

In [3]:
variables = [dimod.Integer("x"),dimod.Integer("y")]

In [4]:
variables

[QuadraticModel({'x': 1.0}, {}, 0.0, {'x': 'INTEGER'}, dtype='float64'),
 QuadraticModel({'y': 1.0}, {}, 0.0, {'y': 'INTEGER'}, dtype='float64')]

In [5]:
variables[0].set_upper_bound("x",2**nx)
variables[1].set_upper_bound("y",2**ny)

In [6]:
cqm = dimod.ConstrainedQuadraticModel()

In [7]:
cqm.set_objective(N - (2*variables[0]+1)*(2*variables[1]+1))

In [8]:
print(cqm)

Constrained quadratic model: 2 variables, 0 constraints, 3 biases

Objective
  220 - 2*Integer('x') - 2*Integer('y') - 4*Integer('x')*Integer('y')

Constraints

Bounds
  0.0 <= Integer('x') <= 8.0
  0.0 <= Integer('y') <= 64.0



In [9]:
cqm.add_constraint(N - (2*variables[0]+1)*(2*variables[1]+1)>=0)

'c418810'

In [10]:
print(cqm)

Constrained quadratic model: 2 variables, 1 constraints, 6 biases

Objective
  220 - 2*Integer('x') - 2*Integer('y') - 4*Integer('x')*Integer('y')

Constraints
  c418810: 220 - 2*Integer('x') - 2*Integer('y') - 4*Integer('x')*Integer('y') >= 0.0

Bounds
  0.0 <= Integer('x') <= 8.0
  0.0 <= Integer('y') <= 64.0



In [11]:
# Definimos el sampler
sampler = LeapHybridCQMSampler()

# Resolvemos el CQM
sampleset = sampler.sample_cqm(cqm)

# Tomamos las soluciones factibles
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)   

In [12]:
best = feasible_sampleset.first.sample
print([2*best[ind]+1 for ind in ["x","y"] ])

[17.0, 13.0]
